In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from scipy import signal
%matplotlib inline
import pyedflib
import numpy as np
#f = pyedflib.EdfReader('/Documents/UC_BERKELEY/Classes/CE_270/EDF winks/Robin-2L-03.10.16.17.17.24.edf')

In [5]:
f = pyedflib.EdfReader('/Documents/UC_BERKELEY/Classes/CE_270/EDF winks/Robin-2L-03.10.16.17.17.24.edf')

IOError: malloc error

In [6]:
dL = pd.read_csv('Robin-1L-03.10.16.17.15.11.csv')
dR = pd.read_csv('Robin-1R-03.10.16.17.10.28.csv')

In [ ]:
#lines = open('../data_real/eeg_zhanna_channels.txt').readlines()
#lines = lines[1:]
#lines = [line.strip() for line in lines]

#CHANNELS = dict()
#for line in lines:
 #   num, _, label = line.partition(' ')
#  num = int(num) - 1
 #    CHANNELS[num] = label

In [ ]:
dL.shape, dR.shape

In [ ]:
dR.head()

In [ ]:
#for column in dR:
    
#    plt.figure()
    #plt.specgram(d[column], NFFT=128, Fs=250, noverlap=64)
#    plt.plot(dR[column])
#    plt.title(column)

In [ ]:
eeg = np.array(dL.ix[:, 0:13])
eeg = eeg[250:77500,]

#tag = np.array(d.ix[:, 'tag'])
#tag = tag[250:]

In [ ]:
from scipy import signal


In [ ]:
# filter from 5 to 35 Hz, helps remove 60Hz noise and replicates paper
## also helps remove the DC line noise (baseline drift)
## 125 is half the sampling rate (250Hz/2)
b, a = signal.butter(4, (2.0/125, 35.0/125), btype='bandpass') 
b, a

In [ ]:
eeg_f = signal.lfilter(b, a, eeg, axis=0)


In [ ]:
## the filter needs a couple samples to converge
## honestly 500 is very conservative, but we don't these samples anyway so whatever
eeg_f = eeg_f[500:]


In [ ]:
plt.figure(figsize=(14, 4))
plt.plot(eeg[500:1000, 7]) ## raw data


In [ ]:
plt.figure(figsize=(14, 4))
plt.plot(eeg_f[:500, 7])

In [ ]:
from sklearn.decomposition import FastICA


In [ ]:
ica = FastICA()
sources = ica.fit_transform(eeg_f)
means = ica.mean_.copy()
mixing = ica.mixing_.copy()

for i in range(ica.components_.shape[0]):
    plt.figure()
    plt.plot(sources[:15000, i])
    plt.title(i)

In [ ]:
eye_blinks_ix = 11
mixing[:, eye_blinks_ix] = 0 # setting eyeblink component to 0
eeg_ff = sources.dot(mixing.T) + means # this is the ICA inverse transform

In [ ]:
plt.figure()
plt.plot(eeg_f[500:15000, 7])

plt.figure()
plt.plot(eeg_ff[:15000, 7])

In [ ]:
import binascii
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
import csv
import numpy as np
from scipy.cluster.vq import kmeans2,whiten

In [ ]:
def somme (a, i1, i2):
    s=0.0
    for i in range(i2+1-i1):
        s=s+a[i1+i]
    return s

In [ ]:
def ploter(a):
    plt.plot(a)
    plt.ylabel('some numbers')
    plt.show()

In [ ]:
g= open('Robin-1L-03.10.16.17.15.11.csv','rb')
gcsv = csv.reader(g, delimiter=',')
l = []
compteur = 0
for row in gcsv:
    if (compteur!=0):
        l.append(float(row[2])+float(row[3])+float(row[11]))
    compteur = compteur + 1

In [ ]:
def chunk(list,length_chunk):
    size = (len(list)/length_chunk)-1
    chunked_list_

In [ ]:
def traiter(l,l_chunk,nb_clusters):
    size = (len(l)/l_chunk)-1
    a = np.zeros((size,l_chunk))
    st= np.zeros((size,1))
    a2 = np.zeros((size,l_chunk))
    st2=np.zeros((size,1))
    
    for i in range(size):
        for j in range(l_chunk):
            a[i,j]= l[i*l_chunk+j]
            a2[i,j]= l[i*l_chunk+j+(l_chunk/2)]
        st[i,0]=somme(a[i],0,l_chunk-1)
        st2[i,0]= somme(a2[i],0,l_chunk-1)
    
    continuum = np.zeros((2*size,1))
    c2=np.zeros((2*size,l_chunk))
    for i in range(size):
        continuum[2*i,0]=st[i,0]
        continuum[2*i+1,0]=st2[i,0]
        c2[2*i]=a[i]
        c2[2*i+1]=a2[i]

    w = whiten(continuum)
    
    k, c = kmeans2(w,nb_clusters,50, minit='points')
    print k
    print c
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.show()
    
    ind_imp= 0
    ou1 = somme(c,0,len(c)-1)
    if (ou1<len(c)/2):
        ind_imp=1
    else:
        ou1=len(c)-ou1
        
    inds = []
    for i in range(len(c)):
        if (c[i]==ind_imp):
            inds.append(i)
    
    print inds
    for i in inds:
        ploter(c2[i])
    return ou1   


In [ ]:

def trouver_l_chunk(l_min,rang):    
    for j in range(5):
        li = []
        for i in range (rang):
            li.append(traiter(l,rang + l_min,2))
        ploter(li)
        return min(li),li.index(min(li))

traiter(l,95,2)
